In [122]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import numpy as np 
import pandas as pd
from sklearn.preprocessing import Imputer

In [123]:
# functions 
def get_mae(max_leaf_nodes, predictors_train, predictors_val, targ_train, targ_val):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(predictors_train, targ_train)
    preds_val = model.predict(predictors_val)
    mae = mean_absolute_error(targ_val, preds_val)
    return mae

def get_mae2(X,y):
    return -1 * cross_val_score(RandomForestRegressor(50), X,y, scoring='neg_mean_absolute_error').mean()


In [124]:
data_file_path = '../input/train.csv'
data = pd.read_csv(data_file_path)
y  = data.SalePrice
x = data.drop(['SalePrice'], axis=1)
x['label'] = 'train'

test_file_path = '../input/test.csv'
test_data = pd.read_csv(test_file_path)
test_data['label'] = 'test'

main_data = pd.concat([x, test_data])
len(main_data)

In [125]:
# removing missing data
cols_missing = [col for col in main_data.columns
                            if main_data[col].isnull().any()]
main_data = main_data.drop(cols_missing, axis=1)
main_data.head()

In [126]:
# one hot encodings
main_data = pd.get_dummies(main_data)
main_data.head()

In [127]:
train_data = main_data[main_data['label_train'] == 1]
test_data = main_data[main_data['label_test'] == 0]

train_data = train_data.drop(['label_train', 'label_test'], axis=1)
test_data = test_data.drop(['label_train', 'label_test'], axis=1)

In [130]:
# split into train_test 
train_x , val_x, train_y, val_y = train_test_split(train_data,y,train_size=0.7,test_size=0.2, random_state=0)

forest_model = RandomForestRegressor()
forest_model.fit(train_x, train_y)
price_preds = forest_model.predict(val_x)
print(mean_absolute_error(val_y, price_preds))

In [140]:
my_imputer = Imputer()
data_file_path = '../input/train.csv'
data = pd.read_csv(data_file_path)
y  = data.SalePrice
x = data.drop(['SalePrice'], axis=1)
x['label'] = 'train'

test_file_path = '../input/test.csv'
test_data = pd.read_csv(test_file_path)
test_data['label'] = 'test'

main_data = pd.concat([x, test_data])
main_data = pd.get_dummies(main_data)

train_data = main_data[main_data['label_train'] == 1]
test_data = main_data[main_data['label_test'] == 0]

train_data = train_data.drop(['label_train', 'label_test'], axis=1)
test_data = test_data.drop(['label_train', 'label_test'], axis=1)
train_x , val_x, train_y, val_y = train_test_split(train_data,y,train_size=0.7,test_size=0.2, random_state=0)


imputed_x_train = my_imputer.fit_transform(train_x)
imputed_x_test = my_imputer.transform(val_x)

forest_model = RandomForestRegressor()
forest_model.fit(imputed_x_train, train_y)
price_preds = forest_model.predict(imputed_x_test)
print(mean_absolute_error(val_y, price_preds))



In [141]:
imputed_test = my_imputer.fit_transform(test_data)

predicted_prices = forest_model.predict(imputed_test)

In [153]:
# make submissions 
my_submission = pd.DataFrame({'Id':test_data.Id, 'SalePrice':predicted_prices})[:-1]
len(my_submission)

In [ ]:
my_submission.to_csv('submission.csv', index=False)
len(my_submission)